In [1]:
"""QMeter Circuit Behavior Simulation
J. Maxwell

Jupyter Notebook Representation of electronic behavior of a series tank Q-meter. Based initially on MathCAD code by
M. Houlden, G. Court. Expanded to include 4 additional transmission lines and test signal.
G.R. Court, M.A. Houlden, et. al. "High precision measurement of the polarization in solid state polarized targets
using NMR," NIM A 527 (2004) 253–263. https://doi.org/10.1016/j.nima.2004.02.041.

"""

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

###### Controls ######
capacitance = 1.0  # tuning capacitance
phase = 0  # phase difference of signal and reference in degrees
trim_sup = 10.  # number of lambda/2 lengths for supply cable
trim_sig = 10.  # number of lambda/2 lengths for signal cable
trim_ref = .5  # number of lambda/2 lengths for reference cable
trim_coil = .00001  # number of lambda/2 lengths for coil cable
sig = True  # include simulated polarization signal?
species = 'p'   # 'p' for protons, 'd' for deuterons
pol = -0.0027   # size of polarization for simulated signal in percent
######################

def update(trim_sup, trim_sig, trim_ref, trim_coil, capacitance, phase, sweep_range):
    u = 1  # Input RF voltage
    r_cc = 681  # Constant current resistor
    i = u / r_cc  # Constant current

    c_stray = 0.001e-12  # Stray capacitance
    # c_stray = 0.001e-12       # Stray capacitance

    # l_0 = 0.055E-6            # Inductance of coil
    l_0 = 0.175E-6  # Inductance of coil
    r_coil = 0.3  # Resistance of coil
    r_amp = 50  # Impedance of detector
    r_source = 50  # Output Impedance of source
    r = 2  # Damping resistor, 2 or 10

    if 'p' in species: 
        gr = 42.576   # Gyromagnetic Ratio in MHz/T, protons 42.576
    elif 'd' in species:    
        gr = 6.53566  # deuterons 6.53566
    else:
        raise ValueError("Species set incorrectly")
        
    b = 5.00  # Holding field in T
    # b = 4.871                 # Holding field in T
    f0 = b * gr  # Larmor freq in MHz
    w0 = 2 * np.pi * f0 * 1e6  # Angular freq

    c = capacitance / (w0 ** 2 * l_0)  # tank capacitance
    print("Tuning capacitance:", c * 1e12, "pf")

    # sweep_range = 5         # frequency sweep range in MHz
    flo = f0 - sweep_range
    fhi = f0 + sweep_range
    wlo = 2 * np.pi * flo * 1e6
    whi = 2 * np.pi * fhi * 1e6

    n_steps = 500  # number of steps in sweep
    dw = (whi - wlo) / n_steps  # step size in freq

    w = lambda k: wlo + k * dw  # sweep frequency in Nsteps of dw
    f = lambda k: w(k) / (2 * np.pi)

    # v_f  = 0.70               # cable velocity factor (flex cable)  (sometimes see this as 0.78 or 0.695)
    v_f = 0.78  # cable velocity factor  (k-flex double shielded)(sometimes see this as 0.78 or 0.695)
    vel = v_f * 2.99e8  # velocity in cable
    lb2 = vel / (2 * f0 * 1e6)  # lambda/2 cable length in m
    len_sup = trim_sup * lb2  # total cable length, factor times lambda/2
    len_sig = trim_sig * lb2
    len_ref = trim_ref * lb2
    len_coil = trim_coil * lb2
    print('Lambda/2 in m:', lb2)
    print('Supply cable length:', trim_sup, "lambda/2, ", len_sup, "m")
    print('Signal cable length:', trim_sig, "lambda/2, ", len_sig, "m")
    print('Reference cable length:', trim_ref, "lambda/2, ", len_ref, "m")
    print('Coil  cable length:', trim_ref, "lambda/2, ", len_coil, "m")

    alph = 0.0343  # attenuation constant in cable
    gam = lambda k: complex(alph, w(k) / vel)  # loss term as signal travels down cable
    r_cab = 2 * 50 * alph  # cable resistance
    l_cab = 2.54e-7  # cable inductance
    c_sup = 95.2e-12 * len_sup  # cable capacitance, per meter
    c_sig = 95.2e-12 * len_sig  # cable capacitance, per meter
    c_ref = 95.2e-12 * len_ref  # cable capacitance, per meter
    c_coil = 95.2e-12 * len_coil  # cable capacitance, per meter

    #### Simulated Signal ####
    chi_0 = 3.1e-4 * pol  # Seely, 1998 proceedings
    del_w = 6e-2 * 2 * np.pi * 1e6  # width bet derivative extrema
    x_of_w = lambda k: (w(k) - w0) / del_w
    chi_pp = lambda k: 1.035 * chi_0 * w0 / del_w * (1 / (1 + 4 * np.power(x_of_w(k), 2.35 + 0j)))  # Hill & Hill
    chi = lambda k: complex(0, chi_pp(k))
    eta = 0.0013  # filling factor

    if (sig):  # include simuilated signal?
        l_coil = lambda k: l_0 * (1 + 4 * np.pi * eta * chi(k))
    else:
        l_coil = lambda k: l_0
    ############################    

    def z0_calc(r_cab, l_cab, c_cab, k):
        '''Characteristic impedance of transmission line, given properties of cable and k to find w'''
        return (np.sqrt((complex(r_cab, w(k) * l_cab)) / (complex(0, w(k) * c_cab))))

    def trans(z0, zl, gamma, length):
        '''Transmission line equation for characteristic impedance z0 and effective load impedance zl, gamma and length'''
        return (z0 * ((zl + z0 * np.tanh(gamma * length)) / (z0 + zl * np.tanh(gamma * length))))

    # z0 = lambda k: np.sqrt((complex(r_cab,w(k)*l_cab))/(complex(0,w(k)*c_cab)))
    z0_sup = lambda k: z0_calc(r_cab, l_cab, c_sup, k)  # characteristic impedance of  supply line
    z0_sig = lambda k: z0_calc(r_cab, l_cab, c_sig, k)  # characteristic impedance of  signal line
    z0_ref = lambda k: z0_calc(r_cab, l_cab, c_ref, k)  # characteristic impedance of  reference line
    z0_coil = lambda k: z0_calc(r_cab, l_cab, c_coil, k)  # characteristic impedance of  coil line

    zc = lambda k: 1 / complex(0, w(k) * c)  # impedance of cap
    zc_stray = lambda k: 1 / complex(0, w(k) * c_stray)  # impedance of stray cap
    zl_pure = lambda k: complex(r_coil, w(k) * l_coil(k))  # impedance of coil only
    z_l = lambda k: zl_pure(k) * zc_stray(k) / (zl_pure(k) + zc_stray(k))  # impedance of coil and stray capacitance
    z_coil = lambda k: trans(z0_coil(k), z_l(k), gam(k), len_coil)  # impedance of coil and it's transmission line
    z_tank = lambda k: r + zc(k) + z_coil(k)  # impedance of tank leg
    z_sup = lambda k: trans(z0_sup(k), r_source, gam(k), len_sup)  # input impedance of supply trans line and source
    z_cc = lambda k: r_cc + 1 / (1 / 50 + 1 / z_sup(
        k))  # impedance of supply line and source, 50 ohm to ground, and constant current resistor

    z_eq = lambda k: 1 / (1 / z_tank(k) + 1 / z_cc(k))
    # z_eq  = lambda k: 1/(1/z_tank(k)+1/z_cc(k)+1/50)                  # impedance of parallel legs of supply trans line and tank
    z_tot = lambda k: trans(z0_sig(k), z_eq(k), gam(k), len_sig)  # impedance of parallel legs and signal trans linetank
    # z_tot = lambda k: trans(z0_sig(k),50,gam(k),len_sig)        # impedance of parallel legs and signal trans line

    z_ref = lambda k: 1 / (1 / trans(z0_ref(k), r_source, gam(k),
                                     len_ref) + 1 / 50)  # input impedance of reference tran line and source
    v_ref = lambda k: u * (r_amp / (r_amp + z_ref(k)))  # voltage into mixer reference

    v_eq = lambda k: u * (z_tank(k) / (z_tank(k) + z_cc(k)))  # voltage into signal line
    v_rf = lambda k: v_eq(k) * (r_amp / (r_amp + z_tot(k)))  # voltage into mixer rf

    phi = phase * np.pi / 180  # phase bet. constant current and output voltage
    irf = lambda k: u * (1000 / (r_cc + z_tot(k)))

    v_out = lambda k: v_rf(k) * np.exp(complex(0, phi))  # voltage into diode
    # v_mixer = lambda k: v_rf(k)*np.exp(complex(0,phi))    # voltage out of ideal mixer

    v_mixer = lambda k: v_rf(k) * v_ref(k) * np.exp(complex(0, phi))  # voltage out of ideal mixer

    # print('Tune capacitance in pF:',c*1e12)
    points = np.arange(0., n_steps, 1)  # make arrays of n_steps points from functions
    v_phase_vec = np.vectorize(v_mixer)
    v_phase = -np.real(v_phase_vec(points))
    v_diode_vec = np.vectorize(v_out)
    v_diode = np.absolute(v_diode_vec(points))

    # print("Freq(0):",f(0)/1e6,"MHz; Freq(499):",f(499)/1e6,"MHz")

    plt.plot(f(points) / 1000000, v_diode)
    # plt.axvline(x=n_steps/2, color='red')
    plt.grid()
    plt.title('Diode Signal')
    plt.xlabel('Freq (MHz)')
    plt.savefig("diode.pdf", bbox_inches='tight')
    plt.show()
    # print("10e5*(",np.absolute(v_out(499)),'-',np.absolute(v_out(0)),')=',10e5*(np.absolute(v_out(499))-np.absolute(v_out(0))))
    plt.plot(f(points) / 1000000, (v_phase))
    # plt.axvline(x=n_steps/2, color='red')
    plt.grid()
    plt.title('Phase Signal')
    plt.xlabel('Freq (MHz)')
    plt.savefig("phase.pdf", bbox_inches='tight')
    plt.show()
    # print("10e5*(",np.real(v_out(499)),'-',np.real(v_out(0)),')=',10e5*(np.real(v_out(499))-np.real(v_out(0))))

    return (v_phase, v_diode)


w = interactive(update, trim_sup=widgets.FloatText(step=0.1, value=0.000000001),
                trim_sig=widgets.FloatText(step=0.1, value=0.000000001),
                trim_ref=widgets.FloatText(step=0.1, value=0.000000001),
                trim_coil=widgets.FloatText(step=0.1, value=0.000000001),
                capacitance=widgets.FloatText(step=0.005, value=1),
                phase=widgets.FloatText(step=1, value=0),
                sweep_range=widgets.FloatText(step=0.1, value=0.4))
output = w.children[-1]
output.layout.height = '1000px'
w

interactive(children=(FloatText(value=1e-09, description='trim_sup', step=0.1), FloatText(value=1e-09, descrip…